# Intro
Notebook to download [Hotels-50K dataset](https://github.com/GWUvision/Hotels-50K) images based on the [download_train.py](https://github.com/GWUvision/Hotels-50K/blob/master/download_train.py) script.

### Disclaimer: We are currently not sure if using external datasets is allowed in this competition!
You can see [Use of External Data Sets](https://www.kaggle.com/competitions/hotel-id-to-combat-human-trafficking-2022-fgvc9/discussion/317922) discussion for details. The competition host did not confirm whether we can use external data yet.

# Download dataset with image info from github repo

In [ ]:
!mkdir hotels-50k
!wget -P hotels-50k https://github.com/GWUvision/Hotels-50K/raw/master/input/dataset.tar.gz
!tar -xvzf hotels-50k/dataset.tar.gz -C hotels-50k
!rm hotels-50k/dataset.tar.gz

# Load data info

In [ ]:
import pandas as pd
import tqdm

In [ ]:
chain_df = pd.read_csv("./hotels-50k/dataset/chain_info.csv")
display(chain_df.head())

In [ ]:
hotel_df = pd.read_csv("./hotels-50k/dataset/hotel_info.csv")
display(hotel_df.head())

In [ ]:
train_df = pd.read_csv('./hotels-50k/dataset/train_set.csv', header=None, 
                       names=['image_id', 'hotel_id', 'url', 'source', 'timestamp'])

display(train_df.head())

# Check Hotels-50k data

In [ ]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
data_df = train_df.merge(hotel_df, on="hotel_id").merge(chain_df, on="chain_id")
data_df["image_id"] = data_df["image_id"].astype(str)
data_df["hotel_id"] = data_df["hotel_id"].astype(str)
data_df["chain_id"] = data_df["chain_id"].astype(str)

display(data_df.head())

### Dataset size

In [ ]:
print("Image count:", len(data_df))
print("Hotel count:", len(data_df["hotel_id"].unique()))
print("Chain count:", len(data_df["chain_id"].unique()))

### Hotel and image count per chain

In [ ]:
chain_group_df = data_df.groupby(["chain_name"]).agg({"hotel_id": [pd.Series.nunique], "image_id" : [pd.Series.nunique]})
chain_group_df.columns = ["_".join(x) for x in chain_group_df.columns.ravel()]
chain_group_df = chain_group_df.reset_index().sort_values("hotel_id_nunique")[::-1]

In [ ]:
fig = px.scatter(chain_group_df, x="chain_name", y="hotel_id_nunique",
                 size="image_id_nunique", color = "image_id_nunique",
                 hover_name = None,
                 log_y=True, size_max=75)

fig.update_yaxes(title_text="Hotel count")
fig.update_xaxes(title_text="Chain ID")
fig.update_layout(title="Hotel and image count per chain", coloraxis=dict(colorbar=dict(title="Image count")))
fig.update_traces(hovertemplate="Chain: %{x} <br>Hotel count: %{y:%d}<br>Image count: %{marker.size:%d}")
fig.show()

### Image count per hotel

In [ ]:
group_df = data_df.groupby(["hotel_id"]).size().to_frame("image_count").sort_values("image_count")[::-1].reset_index()

In [ ]:
fig = px.histogram(group_df, x="image_count", nbins=100, marginal="box", height=500)
fig.update_layout(title="Distribution of image count per hotel")
fig.update_traces(hovertemplate="Image count: %{x} <br>Hotel count: %{y:%d}")
fig.update_yaxes(title_text="Hotel count")
fig.update_xaxes(title_text="Image count")
fig.show()

### Image count per source
Images come from two different sources: travel_website and traffickcam

In [ ]:
group_df = data_df.groupby(["source"]).size().to_frame("image_count").sort_values("image_count")[::-1].reset_index()

fig = px.bar(group_df, x="source", y="image_count", height=500)
fig.update_layout(title="Image count per source")
fig.update_traces(hovertemplate="Source: %{x:%d} <br>Image count: %{y:%d}")
fig.update_yaxes(title_text="Image count")
fig.update_xaxes(title_text="Source")
fig.show()

# Sample 50 hotels with more than 10 and less than 100 images

In [ ]:
hotel_group_df = data_df.groupby(by=["hotel_id"])["image_id"].count().to_frame("image_count")

In [ ]:
sample_hotels = hotel_group_df[(hotel_group_df["image_count"] > 10) & (hotel_group_df["image_count"] < 100)]
print("Number of hotels with more than 10 images and less than 100:", len(sample_hotels))
sample_hotels = sample_hotels.sample(50, random_state=42)

In [ ]:
sample_df = data_df[data_df["hotel_id"].isin(sample_hotels.index)].reset_index(drop=True)
print("Sampled images:", len(sample_df))

In [ ]:
chain_group_df = sample_df.groupby(["chain_name"]).agg({"hotel_id": [pd.Series.nunique], "image_id" : [pd.Series.nunique]})
chain_group_df.columns = ["_".join(x) for x in chain_group_df.columns.ravel()]
chain_group_df = chain_group_df.reset_index().sort_values("hotel_id_nunique")[::-1]

fig = px.scatter(chain_group_df, x="chain_name", y="hotel_id_nunique",
                 size="image_id_nunique", color = "image_id_nunique",
                 hover_name = None,
                 size_max=75)

fig.update_yaxes(title_text="Hotel count")
fig.update_xaxes(title_text="Chain ID")
fig.update_layout(title="Sampled data <br>Hotel and image count per chain", coloraxis=dict(colorbar=dict(title="Image count")))
fig.update_traces(hovertemplate="Chain: %{x} <br>Hotel count: %{y:%d}<br>Image count: %{marker.size:%d}")
fig.show()

# Download sampled images

## Prepare to download images
The SSL certificate of the image urls is expired so we have to handle it.

In [ ]:
from __future__ import print_function
import csv, multiprocessing, cv2, os
import numpy as np
import urllib
import urllib.request

import ssl

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [ ]:
output_folder = "hotels-50k/images"
output_image_folder  = output_folder + "/train"

os.makedirs(output_image_folder)

## Download images

We will download the images without padding or resizing and we will keep the original folder structure: hotels-50k/images/train/chain_id/hotel_id/source/image_id.jpeg

In [ ]:
def url_to_image(url):
    resp = urllib.request.urlopen(url, context=ctx)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_UNCHANGED)
    return image


def download_images(imList):
    # 2d list, rows are samples
    # columns: "chain_id", "hotel_id", "source", "image_id", "url"
    for im in imList:
        try:
            saveDir = os.path.join(output_image_folder, im[0], im[1], im[2])
            if not os.path.exists(saveDir):
                os.makedirs(saveDir)

            savePath = os.path.join(saveDir, str(im[3])+'.'+im[4].split('.')[-1])

            if not os.path.isfile(savePath):
                img = url_to_image(im[4])
                cv2.imwrite(savePath,img)
            else:
                print('Already exists: ' + savePath)
        except Exception as e:
            print(e, ': ' + im[4])

In [ ]:
%%time

image_data_array = sample_df[["chain_id", "hotel_id", "source", "image_id", "url"]].values

pool = multiprocessing.Pool()
NUM_THREADS = multiprocessing.cpu_count()
for cpu in range(NUM_THREADS):
    pool.apply_async(download_images,[image_data_array[cpu::NUM_THREADS]])

pool.close()
pool.join()

Not every image is available, lets check how many images were successfully downloaded

In [ ]:
!find {output_image_folder} -type f | wc -l

## Check downloaded data

In [ ]:
# update the sample data frame with path, image name and whether it was downloaded
sample_df["downloaded"] = False
sample_df["image_name"] = None
sample_df["image_folder"] = None

for index, row in sample_df.iterrows():
    image_folder = os.path.join(output_image_folder, row["chain_id"], row["hotel_id"], row["source"])
    image_name   = row["image_id"] + '.'+ row["url"].split('.')[-1]
    image_path   = os.path.join(image_folder, image_name)
    if os.path.exists(image_path):
        sample_df.loc[index, "downloaded"] = True
        sample_df.loc[index, "image_name"] = image_name
        sample_df.loc[index, "image_folder"] = image_folder

In [ ]:
display(sample_df.head())

In [ ]:
# number of downloaded images should be the same as number of images in the output_image_folder
print("Number of downloaded images:", sample_df["downloaded"].sum())

In [ ]:
# save sample df to csv
sample_df.to_csv("hotels-50k/sample.csv", index=False)

## Display some images

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
downloaded_df = sample_df[sample_df["downloaded"]].sample(6, random_state=42)

fig, axes = plt.subplots(3,2, figsize=(16,20))
axes = axes.ravel()

for i in range(6):
    sample = downloaded_df.iloc[i]
    img = cv2.imread(os.path.join(sample["image_folder"], sample["image_name"])) 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axes[i].imshow(img)
    axes[i].set_title(f"Chain: {sample.chain_name}\n" + 
                     f"Hotel_name: {sample.hotel_name}\n"
                     f"Image: {sample.image_id}\n" +
                     f"Time: {sample.timestamp}\n" + 
                     f"Size: {np.shape(img)}")

# Create zip and clean up
Compress the downloaded images to zip file and delete the data.

In [ ]:
!zip -r -qq hotels-50K-sample.zip hotels-50k
!rm -rf hotels-50k